In [1]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.4 torchvision==0.5 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
Requirement already up-to-date: torch==1.4 in /home/imad/anaconda3/envs/pytorch3d/lib/python3.8/site-packages (1.4.0)
Requirement already up-to-date: torchvision==0.5 in /home/imad/anaconda3/envs/pytorch3d/lib/python3.8/site-packages (0.5.0)
Processing /home/imad/.cache/pip/wheels/52/dd/2b/10ff8b0ac81b93946bb5fb9e6749bae2dac246506c8774e6cf/PyYAML-5.1-cp38-cp38-linux_x86_64.whl
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 5.3.1
ERROR: Cannot uninstall 'PyYAML'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-qsdgtw6t
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-qsdgtw6t
  Created wheel for pycocotools: filename=pycocotools-2.0-cp38-cp38-linux_x86_64.whl size=314049 sha2

In [8]:
!pip install opencv-python

In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
# (add --user if you don't have permission)

# Or, to install it from a local clone:
!git clone https://github.com/facebookresearch/detectron2.git
!python -m pip install -e detectron2

# Or if you are on macOS
# !CC=clang CXX=clang++ python -m pip install -e .

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-1svx508m
  Running command git clone -q https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-1svx508m

In [ ]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import torch 
# import some common libraries
import numpy as np
import cv2
import random
# from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

In [ ]:
import xml.etree.ElementTree as ET
from pprint import pprint
import glob
from detectron2.structures import BoxMode

def get_data_sample(file = '-1x-1',idx=0):


    height, width = cv2.imread('images/'+file+'.jpg').shape[:2]
    record = {}
    
    record["file_name"] = 'images/'+file+'.jpg'
    record["image_id"] = idx
    record["height"] = height
    record["width"] = width

    tree = ET.parse('labels/'+file+'.xml')
    root = tree.getroot()
    objs = []

    for child in root:
        if child.tag == 'object':
            coords = {}
            for child1 in child:

                if child1.tag == 'name':
                    name = child1.text
                if child1.tag == 'bndbox':
                    for child2 in child1:
                        coords[child2.tag] = int(child2.text)
            px = [coords['xmin'], coords['xmin'], coords['xmax'], coords['xmax']]
            py = [coords['ymin'], coords['ymax'], coords['ymax'], coords['ymin']]
            poly = [(x , y) for x, y in zip(px, py)]
            # print(len(poly))
            poly = [p for x in poly for p in x]
            # print(len(poly))
            if name == 'good':
                cat = 0
            elif name == 'bad':
                cat = 1
            else:
                cat = 2
            obj = {
                    "bbox": [coords['xmin'], coords['ymin'], coords['xmax'], coords['ymax']],
                    "bbox_mode": BoxMode.XYXY_ABS,
                    "segmentation": [poly],
                    "category_id": cat,
                    "iscrowd": 0
                }
            objs.append(obj)

    
    record["annotations"] = objs
    return record


def get_all():
    data = [img[:-4].replace('images/','') for img in glob.glob('images/*.jpg')]
    data = [get_data_sample(i) for i in data]
    return data
# get_all()

In [ ]:
# if your dataset is in COCO format, this cell can be replaced by the following three lines:
# from detectron2.data.datasets import register_coco_instances
# register_coco_instances("my_dataset_train", {}, "json_annotation_train.json", "path/to/image/dir")
# register_coco_instances("my_dataset_val", {}, "json_annotation_val.json", "path/to/image/dir")
import os
import numpy as np
import json
from detectron2.structures import BoxMode


from detectron2.data import DatasetCatalog, MetadataCatalog
for d in ["train", "val"]:
    DatasetCatalog.register("mask_" + d, lambda d=d: get_all())
    MetadataCatalog.get("mask_" + d).set(thing_classes=["good","bad","none"])
balloon_metadata = MetadataCatalog.get("mask_train")

In [ ]:
dataset_dicts = get_all()
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=balloon_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("mask_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0025  # pick a good LR
cfg.SOLVER.MAX_ITER = 3000    # 300 iterations seems good enough for this toy dataset; you may need to train longer for a practical dataset
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  # only has one class (ballon)

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
import imp
imp.reload(detectron2.utils.visualizer)
from detectron2.utils.visualizer import Visualizer
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("mask_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0025  # pick a good LR
cfg.SOLVER.MAX_ITER = 3000    # 300 iterations seems good enough for this toy dataset; you may need to train longer for a practical dataset
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  # only has one class (ballon)

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
cfg.DATASETS.TEST = ("mask_val", )
predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.utils.visualizer import ColorMode
# dataset_dicts = get_all()
# for d in random.sample(dataset_dicts, 1): 
def proces_image(im):   
    im = cv2.imread(im)
    outputs = predictor(im)
    v = Visualizer(im,
                    metadata=balloon_metadata, 
                    scale=0.8, 
                    instance_mode=ColorMode.SEGMENTATION   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(v.get_image())
    return True

images = ['000427.jpg', '000457.png', 'example.jpg', 'example2.jpg', 'example4.jpeg']
for img in images:
    proces_image(img)